# RAG 실습

---


####💡 RAG란?

Retrieval-Augmented Generation의 약자로, LLM에게 관련 문서를 먼저 찾아서 (Retrieval) 제공한 후 답변을 생성 (Generation)하게 하는 기법 <br/>이를 통해 최신 정보 활용, 도메인 특화 지식 적용, 그리고 거짓 정보 생성 (Hallucination) 감소 효과를 얻을 수 있습니다.

<br/>

####⭐️ 학습 목표
본 실습에서는 다음을 단계별로 실습합니다:
1. **텍스트 데이터 수집**: Wikipedia에서 다양한 주제의 문서 가져오기
2. **임베딩 이해**: 텍스트를 벡터로 변환하는 과정 체험
3. **벡터DB 저장**: ChromaDB를 사용한 효율적인 검색 시스템 구축
4. **RAG 파이프라인**: 질의 → 검색 → 답변 생성
5. **Hallucination 비교**: RAG가 왜 필요한지 실험으로 증명

---



## 0. 필요한 라이브러리 설치, API 키 설정

In [1]:
!pip install openai==1.54.3 -q
!pip install httpx==0.27.0 -q
!pip install chromadb==0.5.11 -q
!pip install wikipedia-api==0.7.1 -q
!pip install scikit-learn matplotlib -q


In [2]:
import wikipediaapi
import os
os.environ["CHROMA_TELEMETRY_DISABLED"] = "true"
import chromadb
from chromadb.config import Settings
from openai import OpenAI
from typing import List, Dict
import json
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings('ignore')


# openai API key 설정
OPENAI_API_KEY = "YOUR_API_KEY" #TODO: 실제 API키로 교체해주세요
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
client = OpenAI(api_key=OPENAI_API_KEY)


## 1. Wikipedia에서 텍스트 데이터 수집

주제선정: 다양한 도메인에서 데이터를 수집하여 RAG 시스템의 범용성을 테스트
- **유사 주제 그룹 (AI 관련)**: 인공지능, 머신러닝, 딥러닝
- **다양한 도메인**: 프로그래밍, 블록체인, 과학, 환경, 문화, 스포츠, 예술

In [3]:
# Wikipedia API 초기화
wiki = wikipediaapi.Wikipedia(
    user_agent='RAG-Practice/1.0',
    language='ko'
)

# 수집할 주제들
topics = [
    # === 유사 주제: AI 관련 ===
    "인공지능",
    "머신러닝",
    "딥러닝",

    # === 다양한 도메인 ===
    "한국 요리",
    "올림픽",
    "르네상스",
    "파이썬",
    "블록체인",
    "양자 컴퓨팅",
    "기후변화",
]


documents = []

for i, topic in enumerate(topics, 1):
    try:
        page = wiki.page(topic)
        if page.exists():
            # 요약문 사용 (최대 600자)
            summary = page.summary[:600] if len(page.summary) > 600 else page.summary

            documents.append({
                "id": f"doc_{len(documents)}",
                "topic": topic,
                "content": summary,
                "category": "AI" if topic in ["인공지능", "머신러닝", "딥러닝"] else "Other",
                "source": f"https://ko.wikipedia.org/wiki/{topic.replace(' ', '_')}"
            })

            print(f"[{i:2d}/10] '{topic}' 수집 완료 ({len(summary)}자)")
        else:
            print(f"[{i:2d}/10] '{topic}' 페이지를 찾을 수 없습니다.")
    except Exception as e:
        print(f"[{i:2d}/10] '{topic}' 수집 실패: {str(e)}")

print("="*60)
print(f"\n 총 {len(documents)}개의 문서가 수집되었습니다.\n")

[ 1/10] '인공지능' 수집 완료 (349자)
[ 2/10] '머신러닝' 수집 완료 (441자)
[ 3/10] '딥러닝' 수집 완료 (600자)
[ 4/10] '한국 요리' 수집 완료 (600자)
[ 5/10] '올림픽' 수집 완료 (600자)
[ 6/10] '르네상스' 수집 완료 (550자)
[ 7/10] '파이썬' 수집 완료 (382자)
[ 8/10] '블록체인' 수집 완료 (437자)
[ 9/10] '양자 컴퓨팅' 수집 완료 (600자)
[10/10] '기후변화' 수집 완료 (600자)

 총 10개의 문서가 수집되었습니다.



In [4]:
# 수집된 문서 내용 미리보기
for i, doc in enumerate(documents[:3], 1):  # 처음 3개만 표시
    print(f"\n{'='*80}")
    print(f"문서 {i}: {doc['topic']}")
    print(f"내용 미리보기:")
    print(f"{doc['content'][:100]}...")
    print(f"{'='*80}")



문서 1: 인공지능
내용 미리보기:
인공지능(人工智能, 영어: artificial intelligence, AI)은 인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현하려는 컴퓨터 과학의 세부분야 중 하나이다. ...

문서 2: 머신러닝
내용 미리보기:
기계 학습(機械學習) 또는 머신 러닝(영어: machine learning, ML)은 경험을 통해 자동으로 개선하는 컴퓨터 알고리즘의 연구이다. 방대한 데이터를 분석해 '미래를 예...

문서 3: 딥러닝
내용 미리보기:
심층 학습(深層學習) 또는 딥 러닝(영어: deep structured learning, deep learning 또는 hierarchical learning)은 여러 '비선형 변...


## 2. 임베딩 생성

- 임베딩은 텍스트를 숫자 벡터로 변환하는 과정입니다.
- 컴퓨터는 텍스트를 직접 이해할 수 없기 때문에, 숫자 벡터로 변환해 수학적으로 유사도를 계산할 수 있어야 합니다.
- 의미가 비슷한 문장들은 벡터 공간에서 가까이 위치합니다.

In [5]:
from typing_extensions import Text
def get_embedding(text: str, model="text-embedding-3-small") -> List[float]:
    """
    OpenAI API를 사용하여 텍스트를 임베딩 벡터로 변환

    Args:
        text: 임베딩할 텍스트
        model: 사용할 임베딩 모델

    Returns:
        1536차원의 벡터 (실수 리스트)
    """

    try:
        response = client.embeddings.create(
            input=text,
            model=model
        )
        return response.data[0].embedding

    except Exception as e:
        print(f"임베딩 생성 실패: {str(e)}")
        return None


In [6]:
# 모든 문서에 대해 임베딩 생성
for i, doc in enumerate(documents, 1):
  embedding = get_embedding(doc["content"])
  if embedding:
      doc["embedding"] = embedding
      print(f"[{i:2d}/{len(documents)}] '{doc['topic']}' 임베딩 완료 (차원: {len(embedding)})")

documents = [doc for doc in documents if 'embedding' in doc]


[ 1/10] '인공지능' 임베딩 완료 (차원: 1536)
[ 2/10] '머신러닝' 임베딩 완료 (차원: 1536)
[ 3/10] '딥러닝' 임베딩 완료 (차원: 1536)
[ 4/10] '한국 요리' 임베딩 완료 (차원: 1536)
[ 5/10] '올림픽' 임베딩 완료 (차원: 1536)
[ 6/10] '르네상스' 임베딩 완료 (차원: 1536)
[ 7/10] '파이썬' 임베딩 완료 (차원: 1536)
[ 8/10] '블록체인' 임베딩 완료 (차원: 1536)
[ 9/10] '양자 컴퓨팅' 임베딩 완료 (차원: 1536)
[10/10] '기후변화' 임베딩 완료 (차원: 1536)


(참고) 임베딩 벡터 출력해보기

In [7]:
def format_vec(vec, n=20, precision=6, show_sign=False):
    """리스트를 문자열로 잘라서 표시"""
    if show_sign:
        fmt = lambda v: f"{v:+.{precision}f}"
    else:
        fmt = lambda v: f"{v:.{precision}f}"
    head = ", ".join(fmt(v) for v in vec[:n])
    tail = " ..." if len(vec) > n else ""
    return "[" + head + tail + "]"

# 여러 개 미리보기 (예: 상위 5개 문서)
k = min(5, len(documents))
for i, doc in enumerate(documents[:k], 1):
    vec = doc.get("embedding")
    if not vec:
        print(f"[{i}] {doc.get('topic','(no topic)')} → embedding 없음")
        continue
    print(f"[{i}] {doc['topic']} | dim={len(vec)}")
    print(format_vec(vec, n=20, precision=6, show_sign=False))
    print("-" * 70)


[1] 인공지능 | dim=1536
[0.020815, 0.024216, -0.006495, 0.021805, 0.027076, -0.052677, -0.005375, 0.057031, 0.009706, 0.006252, 0.018063, -0.013952, -0.023964, -0.048108, 0.029847, -0.029397, -0.046309, -0.016938, 0.068437, -0.027832 ...]
----------------------------------------------------------------------
[2] 머신러닝 | dim=1536
[-0.007226, 0.033005, -0.008111, -0.012604, 0.045193, -0.023912, 0.022945, 0.031999, 0.001929, 0.030702, 0.006099, -0.032966, -0.045851, -0.003195, 0.017963, 0.005509, 0.006109, -0.024280, 0.028729, -0.015719 ...]
----------------------------------------------------------------------
[3] 딥러닝 | dim=1536
[-0.006711, -0.004804, -0.025430, 0.004809, 0.013380, -0.047315, 0.015026, 0.060526, -0.059766, 0.019658, 0.021674, -0.003305, -0.022075, -0.012135, 0.056559, -0.017738, 0.017622, 0.001696, 0.031783, -0.017242 ...]
----------------------------------------------------------------------
[4] 한국 요리 | dim=1536
[-0.018430, 0.018691, -0.038127, -0.009555, 0.053270, -0.044292

(참고) 임베딩 간 유사도 계산하기

In [8]:
def cosine_similarity(vec1: List[float], vec2: List[float]) -> float:
    """
    두 벡터의 코사인 유사도 계산

    Returns:
        -1 ~ 1 사이의 값 (1에 가까울수록 유사)
    """
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


print("문서 간 유사도 비교\n")

# AI 관련 문서들끼리의 유사도
ai_docs = [doc for doc in documents if doc["category"] == "AI"]
if len(ai_docs) >= 2:
    print("AI 관련 문서들 (유사한 주제)")
    print("-"*80)
    for i in range(len(ai_docs)):
        for j in range(i+1, len(ai_docs)):
            sim = cosine_similarity(ai_docs[i]["embedding"], ai_docs[j]["embedding"])
            print(f"   '{ai_docs[i]['topic']}' <-> '{ai_docs[j]['topic']}':    유사도: {sim:.4f}")
    print("")

# 서로 다른 카테고리 문서들의 유사도
other_docs = [doc for doc in documents if doc["category"] != "AI"][:7]
if len(ai_docs) > 0 and len(other_docs) > 0:
    print("AI vs 다른 주제 (서로 다른 주제)")
    print("-"*80)
    for i in range(min(2, len(ai_docs))):
        for j in range(min(3, len(other_docs))):
            sim = cosine_similarity(ai_docs[i]["embedding"], other_docs[j]["embedding"])
            print(f"   '{ai_docs[i]['topic']}' <-> '{other_docs[j]['topic']}:    유사도: {sim:.4f}'")


문서 간 유사도 비교

AI 관련 문서들 (유사한 주제)
--------------------------------------------------------------------------------
   '인공지능' <-> '머신러닝':    유사도: 0.4972
   '인공지능' <-> '딥러닝':    유사도: 0.3810
   '머신러닝' <-> '딥러닝':    유사도: 0.5580

AI vs 다른 주제 (서로 다른 주제)
--------------------------------------------------------------------------------
   '인공지능' <-> '한국 요리:    유사도: 0.0735'
   '인공지능' <-> '올림픽:    유사도: 0.0975'
   '인공지능' <-> '르네상스:    유사도: 0.0607'
   '머신러닝' <-> '한국 요리:    유사도: 0.1451'
   '머신러닝' <-> '올림픽:    유사도: 0.2056'
   '머신러닝' <-> '르네상스:    유사도: 0.0751'


## 3. VectorDB 저장
- 일반 데이터베이스는 텍스트 검색만 가능하지만, 벡터디비는
  1. 의미적으로 유사한 문서를 빠르게 찾을 수 있습니다.
  2. 수백만개의 벡터 중에서도 효율적으로 검색합니다.
  3. 메타데이터(출처) 등도 함께 저장할 수 있습니다.

In [9]:
# ChromaDB 클라이언트 초기화
from chromadb.config import Settings
import logging
logging.getLogger("chromadb.telemetry").setLevel(logging.CRITICAL)
logging.getLogger("chromadb.telemetry.product.posthog").setLevel(logging.CRITICAL)

chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
))

# 이전 컬렉션이 있다면 삭제
try:
    chroma_client.delete_collection(name="rag_practice")
except:
    pass

# 새 컬렉션 생성
collection = chroma_client.create_collection(
    name="rag_practice",
    metadata={"description": "RAG 실습을 위한 Wikipedia 문서 컬렉션"}
)


if len(documents) > 0:
    # 벡터와 메타데이터 저장
    collection.add(
        ids=[doc["id"] for doc in documents],
        embeddings=[doc["embedding"] for doc in documents],
        documents=[doc["content"] for doc in documents],
        metadatas=[
            {
                "topic": doc["topic"],
                "category": doc["category"],
                "source": doc["source"]
            }
            for doc in documents
        ]
    )

    print(f"컬렉션 이름: {collection.name}")
    print(f"저장된 문서 수: {collection.count()}")
    print(f"메타데이터: topic, category, source")
else:
    print("저장할 문서가 없습니다.")


컬렉션 이름: rag_practice
저장된 문서 수: 10
메타데이터: topic, category, source


(참고) Chunking 크기 선택 기준

- chunking이 필요한 이유
  1. **LLM의 입력 제한**: 한 번에 처리할 수 있는 토큰 수가 제한되어 있습니다
  2. **검색 정확도**: 작은 조각이 더 구체적인 정보를 담고 있어 검색이 정확합니다
  3. **비용 효율**: 필요한 부분만 LLM에 전달하여 비용을 절감합니다

- chunking 크기 선택 기준

| Chunk 크기 | 장점 | 단점 | 적합한 경우 |
|-----------|------|------|------------|
| **작음** (100-300자) | 검색 정확도 높음<br>비용 효율적 | 문맥 손실 가능<br>너무 많은 chunk | 구체적인 사실 검색<br>(예: Q&A, FAQ) |
| **중간** (300-600자) | 균형잡힌 선택<br>문맥 유지 | - | **대부분의 경우 권장** |
| **큼** (600-1000자) | 문맥 유지 잘 됨 | 검색 정확도 하락<br>비용 증가 | 문맥이 중요한 경우<br>(예: 소설, 논문) |

## 4. RAG Pipeline 구현


```
1. 사용자 질의
      ⬇️
2. 질의를 임베딩으로 변환
      ⬇️
3. Vector DB에서 유사한 문서 검색 (Top-K)
      ⬇️
4. 검색된 문서를 컨텍스트로 LLM에 전달
      ⬇️
5. LLM이 컨텍스트 기반 답변 생성
```


In [15]:
def rag_query(query: str, top_k: int = 3, show_details: bool = True):
    """
    RAG 시스템을 사용하여 질의에 답변
    """
    if collection.count() == 0:
        print("Vector DB에 문서가 없습니다.")
        return None

    if show_details:
        print(f"\n{'='*80}")
        print(f"사용자 질의: {query}")
        print(f"{'='*80}\n")

    # STEP 1: 질의 임베딩 생성
    if show_details:
        print("1. Query를 벡터로 변환 중...")

    query_embedding = get_embedding(query)
    if not query_embedding:
        print("질의 임베딩 생성 실패")
        return None

    if show_details:
        print(f"   임베딩 완료 (차원: {len(query_embedding)})\n")

    # STEP 2: Vector DB에서 유사 문서 검색
    if show_details:
        print(f"2️. Vector DB에서 Top-{top_k} 문서 검색 중...")

    try:
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=min(top_k, collection.count())
        )
    except Exception as e:
        print(f" 검색 실패: {str(e)}")
        return None

    if not results['documents'][0]:
        print(" 검색 결과가 없습니다.")
        return None

    if show_details:
        print(f"   {len(results['documents'][0])}개 문서 발견\n")
        print("   검색 결과:")
        print("   " + "-"*76)

        for i, (doc, metadata, distance) in enumerate(zip(
            results['documents'][0],
            results['metadatas'][0],
            results['distances'][0]
        ), 1):
            print(f"\n   [{i}위] {metadata.get('topic', 'Unknown')}")
            print(f"        카테고리: {metadata.get('category', 'Unknown')}")
            print(f"        유사도 거리: {distance:.4f}")
            print(f"        내용: {doc[:100]}...")

        print("\n   " + "-"*76)

    # STEP 3: 컨텍스트 구성
    context = "\n\n".join([
        f"[문서 {i+1}: {metadata.get('topic', 'Unknown')}]\n{doc}"
        for i, (doc, metadata) in enumerate(zip(
            results['documents'][0],
            results['metadatas'][0]
        ))
    ])

    # STEP 4: LLM 답변 생성
    if show_details:
        print("\n3. LLM이 답변 생성 중...\n")

    try:
        prompt = f"""다음은 질문에 답변하는데 참고할 수 있는 문서들입니다:

{context}

위 문서들을 참고하여 다음 질문에 답변해주세요.
답변할 때는 어떤 문서를 참고했는지 언급해주세요.

질문: {query}

답변은 명확하고 간결하게 한국어로 작성해주세요."""

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "당신은 주어진 문서를 정확히 바탕으로 답변하는 전문 도우미입니다."
                },
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=500
        )

        answer = response.choices[0].message.content

        if show_details:
            print(f"{'='*80}")
            print("LLM 답변")
            print(f"{'='*80}")
            print(answer)
            print(f"\n{'='*80}\n")

        return {
            "query": query,
            "retrieved_docs": results,
            "answer": answer,
            "context": context
        }
    except Exception as e:
        print(f"LLM 답변 생성 실패: {str(e)}")
        return None

## 5. RAG vs No-RAG 실험

In [16]:
# 실험용 질문
test_question = "머신러닝과 딥러닝의 구체적인 차이점을 설명해주세요."

### 1) No-RAG

In [17]:
def no_rag_query(query: str):
    """RAG 없이 LLM에게 직접 질문"""
    print(f"\n{'='*80}")
    print(f"NO-RAG")
    print(f"{'='*80}")
    print(f"질문: {query}\n")

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
                {"role": "user", "content": query}
            ],
            temperature=0.3,
            max_tokens=500
        )

        answer = response.choices[0].message.content
        print("답변:")
        print("-"*80)
        print(answer)
        print("="*80)
        return answer
    except Exception as e:
        print(f"답변 생성 실패: {str(e)}")
        return None


no_rag_answer = no_rag_query(test_question)


NO-RAG
질문: 머신러닝과 딥러닝의 구체적인 차이점을 설명해주세요.

답변:
--------------------------------------------------------------------------------
머신러닝(Machine Learning)과 딥러닝(Deep Learning)은 인공지능(AI) 분야에서 중요한 두 가지 개념입니다. 이 둘은 서로 관련이 있지만, 몇 가지 중요한 차이점이 있습니다.

### 1. 정의
- **머신러닝**: 머신러닝은 데이터에서 패턴을 학습하고 예측을 수행하는 알고리즘의 집합입니다. 머신러닝은 주로 특징(feature)을 수동으로 선택하고, 이를 기반으로 모델을 학습합니다.
- **딥러닝**: 딥러닝은 머신러닝의 한 하위 분야로, 인공신경망(Artificial Neural Networks)을 기반으로 합니다. 딥러닝은 여러 층의 신경망을 사용하여 데이터에서 자동으로 특징을 추출합니다.

### 2. 데이터 처리
- **머신러닝**: 머신러닝 알고리즘은 일반적으로 구조화된 데이터에 잘 작동하며, 특징 선택 및 전처리가 중요합니다. 예를 들어, SVM(Support Vector Machine), 결정 트리(Decision Tree), 랜덤 포레스트(Random Forest) 등이 있습니다.
- **딥러닝**: 딥러닝은 비구조화된 데이터(예: 이미지, 텍스트, 음성 등)를 처리하는 데 강력합니다. 신경망은 데이터의 복잡한 패턴을 자동으로 학습할 수 있습니다.

### 3. 모델의 복잡성
- **머신러닝**: 머신러닝 모델은 상대적으로 간단하며, 일반적으로 몇 개의 층과 파라미터로 구성됩니다. 따라서 학습 속도가 빠르고, 해석이 용이합니다.
- **딥러닝**: 딥러닝 모델은 수십 개에서 수백 개의 층을 가질 수 있으며, 매우 복잡한 구조를 가지고 있습니다. 이로 인해 대량의 데이터와 계산 자원이 필요합니다.

### 4. 학습 데이터의 양
- **머신러닝**: 머신러닝 알고리즘은 상대적으로 적은 양의 데이터로도 

### 2) With RAG

In [18]:
rag_result = rag_query(test_question, top_k=3, show_details=True)


사용자 질의: 머신러닝과 딥러닝의 구체적인 차이점을 설명해주세요.

1. Query를 벡터로 변환 중...
   임베딩 완료 (차원: 1536)

2️. Vector DB에서 Top-3 문서 검색 중...
   3개 문서 발견

   검색 결과:
   ----------------------------------------------------------------------------

   [1위] 딥러닝
        카테고리: AI
        유사도 거리: 1.3589
        내용: 심층 학습(深層學習) 또는 딥 러닝(영어: deep structured learning, deep learning 또는 hierarchical learning)은 여러 '비선형 변...

   [2위] 머신러닝
        카테고리: AI
        유사도 거리: 1.4972
        내용: 기계 학습(機械學習) 또는 머신 러닝(영어: machine learning, ML)은 경험을 통해 자동으로 개선하는 컴퓨터 알고리즘의 연구이다. 방대한 데이터를 분석해 '미래를 예...

   [3위] 양자 컴퓨팅
        카테고리: Other
        유사도 거리: 1.5997
        내용: 양자 컴퓨터(quantum computer, 문화어: 량자 콤퓨터)는 얽힘(entanglement)이나 중첩(superposition) 같은 양자역학적인 현상을 활용하여 자료를 처...

   ----------------------------------------------------------------------------

3. LLM이 답변 생성 중...

LLM 답변
머신러닝과 딥러닝의 구체적인 차이점은 다음과 같습니다.

1. **정의**:
   - 머신러닝(문서 2): 경험을 통해 자동으로 개선하는 컴퓨터 알고리즘의 연구로, 방대한 데이터를 분석하여 미래를 예측하는 기술입니다.
   - 딥러닝(문서 1): 여러 비선형 변환기법의 조합을 통해 